In [1]:
#| hide
from l402_python.core import *

# l402-python

> 

This file will become your README and also the index of your documentation.

## Developer Guide

If you are new to using `nbdev` here are some useful pointers to get you started.

### Install l402_python in Development mode

```sh
# make sure l402_python package is installed in development mode
$ pip install -e .

# make changes under nbs/ directory
# ...

# compile to have changes apply to l402_python
$ nbdev_prepare
```

## Usage

### Installation

Install latest from the GitHub [repository][repo]:

```sh
$ pip install git+https://github.com/Fewsats/l402-python.git
```

or from [conda][conda]

```sh
$ conda install -c Fewsats l402_python
```

or from [pypi][pypi]


```sh
$ pip install l402_python
```


[repo]: https://github.com/Fewsats/l402-python
[docs]: https://Fewsats.github.io/l402-python/
[pypi]: https://pypi.org/project/l402-python/
[conda]: https://anaconda.org/Fewsats/l402-python

### Documentation

Documentation can be found hosted on this GitHub [repository][repo]'s [pages][docs]. Additionally you can find package manager specific guidelines on [conda][conda] and [pypi][pypi] respectively.

[repo]: https://github.com/Fewsats/l402-python
[docs]: https://Fewsats.github.io/l402-python/
[pypi]: https://pypi.org/project/l402-python/
[conda]: https://anaconda.org/Fewsats/l402-python

## How to use

Fill me in please! Don't forget code examples:

In [2]:
#| hide
import asyncio, socket, time
from threading import Thread
import uvicorn
from fastapi.responses import JSONResponse
from fastapi import Request


In [3]:

def is_port_free(port, host='localhost'):
    "Check if port is available"
    sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    try:
        sock.bind((host, port))
        return True
    except OSError: return False
    finally: sock.close()

def wait_port_free(port, host='localhost', max_wait=3):
    "Wait for port to become available"
    start = time.time()
    while not is_port_free(port):
        if time.time() - start > max_wait: return False
        time.sleep(0.1)
    return True

class ServerManager:
    def __init__(self, app, port=8000, host='0.0.0.0'):
        self.server = uvicorn.Server(uvicorn.Config(app, host=host, port=port, log_level="error"))
        
    def start(self):
        Thread(target=lambda: asyncio.run(self.server.serve()), daemon=True).start()
        while not self.server.started: time.sleep(0.01)
        return self
        
    def stop(self):
        if self.server.started:
            self.server.should_exit = True
            wait_port_free(self.server.config.port)
        return True

In [4]:
from fastapi import FastAPI

# Create FastAPI app
app = FastAPI()

# Add some test endpoint
@app.get("/")
def read_root(request):
    return {"Hello": "World"}

server = ServerManager(app).start()

In [5]:
class PaymentServer:
    def __init__(self):
        pass

    def create_offers(self, offer_id, title, description, type, amount, currency, payment_methods):
        payment_context_token = "b79040aa-94cc-49e7-8894-c33a5699f5d1"
        payment_request_url = "https://localhost:8000"
        return {
            "offers": [{
                "amount": 1,
                "balance": 1,
                "currency": "USD",
                "description": "Purchase 1 credit for API access",
                "offer_id": "offer_c668e0c0",
                "payment_methods": ["lightning"],
                "title": "1 Credit Package",
                "type": "top-up"
            }],
            "payment_context_token": payment_context_token,
            "payment_request_url": payment_request_url,
            "terms_url": "https://link-to-terms.com",
            "version": "0.2.1"
        }
    
    def create_payment_request(self, offer_id, payment_context_token, payment_method):
        expires_at = (datetime.utcnow() + timedelta(days=1)).isoformat() + "+00:00"
        
        # Example for Lightning payment
        if payment_method == "lightning":
            return {
                "expires_at": expires_at,
                "offer_id": "offer_c668e0c0",
                "payment_request": {
                    "lightning_invoice": "lnbc100n1pnk0mkhpp59f0hhld9nu55u2w0hyczlvt00fa5xryrrgrs9qcrp32xznsh6cyqdq6xysyxun9v35hggzsv93kkct8v5cqzpgxqrzpnrzjqwghf7zxvfkxq5a6sr65g0gdkv768p83mhsnt0msszapamzx2qvuxqqqqz99gpz55yqqqqqqqqqqqqqq9qrzjq25carzepgd4vqsyn44jrk85ezrpju92xyrk9apw4cdjh6yrwt5jgqqqqz99gpz55yqqqqqqqqqqqqqq9qsp5ccrauz34377k9yttjqnt366efp6fz27wa47954dxmj0s769e9x6q9qxpqysgqg8fw4qsxe7qjjul2cmhxk54a4yqchg3lpr3tq73ta05wzm3xc96h3sxv9hvennyw2erzrfd92s47sqhencflnxtzxdtnn4n2gxrgsmcp02nv9e"
                },
                "version": "0.2.1"
            }
        # Example for Stripe payment
        elif payment_method == "stripe":
            return {
                "expires_at": expires_at,
                "offer_id": "offer_a896b13c",
                "payment_request": {
                    "checkout_url": "https://checkout.stripe.com/c/pay/cs_test_a1uDnMH9mY1gW1U3RUgkzNilQcY7CaQuo7vxOzbJH4yxY1SabXoGF6qQWo#fidkdWxOYHwnPyd1blpxYHZxWjA0S3VzXDdBbTFNVlJzfDVRT2pxd3AyandqTmAzSEdMYDZUa05yY1VKNWZOUXNVc09ITUdjUGNUUjB2TTVtUkBmVGxmSD1qN0JpZkBuTlMxNm1LcVFtdDVQNTVEc21jdWtoTCcpJ2N3amhWYHdzYHcnP3F3cGApJ2lkfGpwcVF8dWAnPyd2bGtiaWBabHFgaCcpJ2BrZGdpYFVpZGZgbWppYWB3dic%2FcXdwYHgl"
                },
                "version": "0.2.1"
            }
        # Example for Coinbase payment
        else:
            return {
                "expires_at": expires_at,
                "offer_id": "offer_c668e0c0",
                "payment_request": {
                    "checkout_url": "https://commerce.coinbase.com/pay/2a5d6d11-0797-494a-8058-d62efef7a6ef",
                    "contract_addresses": {
                        "1": "0x1FA57f879417e029Ef57D7Ce915b0aA56A507C31",
                        "137": "0x288844216a63638381784E0C1081A3826fD5a2E4",
                        "8453": "0x03059433BCdB6144624cC2443159D9445C32b7a8"
                    }
                },
                "version": "0.2.1"
            }

ps = PaymentServer()

In [ ]:
class PaymentServer:
    def __init__(self):
        pass

    def create_offers(self, offer_id, title, description, type, amount, currency, payment_methods):
        payment_context_token = "b79040aa-94cc-49e7-8894-c33a5699f5d1"
        payment_request_url = "https://localhost:8000"
        return {
            "offers": [{
                "amount": 1,
                "balance": 1,
                "currency": "USD",
                "description": "Purchase 1 credit for API access",
                "offer_id": "offer_c668e0c0",
                "payment_methods": ["lightning"],
                "title": "1 Credit Package",
                "type": "top-up"
            }],
            "payment_context_token": payment_context_token,
            "payment_request_url": payment_request_url,
            "terms_url": "https://link-to-terms.com",
            "version": "0.2.1"
        }
    
    def create_payment_request(self, offer_id, payment_context_token, payment_method, expires_at):
        
        # Example for Lightning payment
        if payment_method == "lightning":
            return {
                "expires_at": expires_at,
                "offer_id": "offer_c668e0c0",
                "payment_request": {
                    "lightning_invoice": "lnbc100n1pnk0mkhpp59f0hhld9nu55u2w0hyczlvt00fa5xryrrgrs9qcrp32xznsh6cyqdq6xysyxun9v35hggzsv93kkct8v5cqzpgxqrzpnrzjqwghf7zxvfkxq5a6sr65g0gdkv768p83mhsnt0msszapamzx2qvuxqqqqz99gpz55yqqqqqqqqqqqqqq9qrzjq25carzepgd4vqsyn44jrk85ezrpju92xyrk9apw4cdjh6yrwt5jgqqqqz99gpz55yqqqqqqqqqqqqqq9qsp5ccrauz34377k9yttjqnt366efp6fz27wa47954dxmj0s769e9x6q9qxpqysgqg8fw4qsxe7qjjul2cmhxk54a4yqchg3lpr3tq73ta05wzm3xc96h3sxv9hvennyw2erzrfd92s47sqhencflnxtzxdtnn4n2gxrgsmcp02nv9e"
                },
                "version": "0.2.1"
            }
        # Example for Stripe payment
        elif payment_method == "stripe":
            return {
                "expires_at": expires_at,
                "offer_id": "offer_a896b13c",
                "payment_request": {
                    "checkout_url": "https://checkout.stripe.com/c/pay/cs_test_a1uDnMH9mY1gW1U3RUgkzNilQcY7CaQuo7vxOzbJH4yxY1SabXoGF6qQWo#fidkdWxOYHwnPyd1blpxYHZxWjA0S3VzXDdBbTFNVlJzfDVRT2pxd3AyandqTmAzSEdMYDZUa05yY1VKNWZOUXNVc09ITUdjUGNUUjB2TTVtUkBmVGxmSD1qN0JpZkBuTlMxNm1LcVFtdDVQNTVEc21jdWtoTCcpJ2N3amhWYHdzYHcnP3F3cGApJ2lkfGpwcVF8dWAnPyd2bGtiaWBabHFgaCcpJ2BrZGdpYFVpZGZgbWppYWB3dic%2FcXdwYHgl"
                },
                "version": "0.2.1"
            }
        # Example for Coinbase payment
        else:
            return {
                "expires_at": expires_at,
                "offer_id": "offer_c668e0c0",
                "payment_request": {
                    "checkout_url": "https://commerce.coinbase.com/pay/2a5d6d11-0797-494a-8058-d62efef7a6ef",
                    "contract_addresses": {
                        "1": "0x1FA57f879417e029Ef57D7Ce915b0aA56A507C31",
                        "137": "0x288844216a63638381784E0C1081A3826fD5a2E4",
                        "8453": "0x03059433BCdB6144624cC2443159D9445C32b7a8"
                    }
                },
                "version": "0.2.1"
            }

ps = PaymentServer()

In [6]:
from pydantic import BaseModel
from typing import List, Optional, Dict


class OfferRequest(BaseModel):
    offer_id: str
    title: str
    description: str
    type: str
    amount: float
    currency: str
    payment_methods: List[str]

class Offer(BaseModel):
    amount: int
    balance: int
    currency: str
    description: str
    offer_id: str
    payment_methods: List[str]
    title: str
    type: str

class OfferResponse(BaseModel):
    offers: List[Offer]
    payment_context_token: str
    payment_request_url: str
    terms_url: str
    version: str

@app.post("/offers")
def offers(request: OfferRequest):
    offers = ps.create_offers(**request.model_dump())
    return JSONResponse(
        content=OfferResponse(**offers).model_dump(),
        status_code=402
    )

In [7]:
class PaymentRequest(BaseModel):
    offer_id: str
    payment_context_token: str
    payment_method: str
    expires_at: str # ??
    
class PaymentMethodDetails(BaseModel):
    lightning_invoice: Optional[str] = None
    checkout_url: Optional[str] = None
    contract_addresses: Optional[Dict[str, str]] = None

class PaymentResponse(BaseModel):
    expires_at: str
    offer_id: str
    payment_request: PaymentMethodDetails
    version: str
    

@app.post("/payment_request")
async def create_payment_request(request: PaymentRequest):
    payment_request = ps.create_payment_request(**request.model_dump())
    return JSONResponse(
        content=payment_request,
        status_code=200
    )


In [8]:
import httpx

# Test offers endpoint
offer_data = {
    "offer_id": "123",
    "title": "Test Offer",
    "description": "Test Description",
    "type": "standard",
    "amount": 100.0,
    "currency": "USD",
    "payment_methods": ["bitcoin", "lightning"]
}

r1 = httpx.post("http://localhost:8000/offers", json=offer_data)
r1, r1.json()


(<Response [402 Payment Required]>,
 {'offers': [{'amount': 1,
    'balance': 1,
    'currency': 'USD',
    'description': 'Purchase 1 credit for API access',
    'offer_id': 'offer_c668e0c0',
    'payment_methods': ['lightning'],
    'title': '1 Credit Package',
    'type': 'top-up'}],
  'payment_context_token': 'b79040aa-94cc-49e7-8894-c33a5699f5d1',
  'payment_request_url': 'https://localhost:8000',
  'terms_url': 'https://link-to-terms.com',
  'version': '0.2.1'})

In [9]:
print(f"Offers Response: {r1.status_code}")
print(r1.json())

# Test payment request endpoint
payment_data = {
    "offer_id": "123",
    "payment_context_token": "token123",
    "payment_method": "lightning"
}

r2 = httpx.post("http://localhost:8000/payment_request", json=payment_data)
print(f"Payment Response: {r2.status_code}")
print(r2.json())

Offers Response: 402
{'offers': [{'amount': 1, 'balance': 1, 'currency': 'USD', 'description': 'Purchase 1 credit for API access', 'offer_id': 'offer_c668e0c0', 'payment_methods': ['lightning'], 'title': '1 Credit Package', 'type': 'top-up'}], 'payment_context_token': 'b79040aa-94cc-49e7-8894-c33a5699f5d1', 'payment_request_url': 'https://localhost:8000', 'terms_url': 'https://link-to-terms.com', 'version': '0.2.1'}
Payment Response: 200
None
